## Goals
* Identify logical areas for future phased expansion of the pilot program.
* Explore the calls data to look for any trends in terms of times or locations. Are there any precincts that get more than their share of calls?
* Determine if there are any trends that may be disproportionality responding to populations of high poverty, low education, housing inequities, or race/ethnicity.


In [ ]:
import pandas as pd
import sqlite3 as sql

Connect to Census Data

In [ ]:
conn = sql.connect(r"\Users\mnorm\Documents\NSS\Python\Projects\partners_in_care-rasberry-pi\data\census.sqlite")
cur = conn.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables = (cur.fetchall())
tables

Disconnect from Census Data

In [ ]:
cur.close()
conn.close()

Import Police Call Data

In [ ]:
file = pd.ExcelFile(r"\Users\mnorm\Documents\NSS\Python\Projects\partners_in_care-rasberry-pi\data\Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx")

In [ ]:
incidents_df = pd.read_excel(file, 'incident_types')
incidents_df.head(3)

In [ ]:
priority_df = pd.read_excel(file, 'priority')
priority_df.head(3)

In [ ]:
calls_df = pd.read_excel(file, 'call_info')
calls_df.head(3)